<a href="https://colab.research.google.com/github/faustoman28/dl-survey-live-lab-2025/blob/main/notebooks/01_baseline_random_forest_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 — Baseline Random Forest (Student)

**Today we will:**
1) Load the Adult dataset (classification: `income`).
2) Split train/test **first** (avoid leakage).
3) Handle missing values (numeric: mean; categorical: most frequent).
4) Encode categorical features:
   - Label encode **one** column (`sex`)
   - One-hot encode the rest
5) (Practice) Scale numeric features (StandardScaler).
6) Train a **RandomForestClassifier** and evaluate it.
7) **Deep dive** into the RF object: attributes & methods.
8) Try a few hyperparameters and record results in your `students/experiment_log.md`.
9) Peek at **GridSearchCV** (teacher-led demo).

> As you work: add short notes in your experiment log (Goal → Setup → Results → Reflection).


### Setup

In [1]:
# 0) Setup
!pip -q install scikit-learn pandas matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import fetch_openml

print("Ready.")


Ready.


### Load data

In [ ]:
# 1) Load the Adult dataset from OpenML
adult = fetch_openml(name="adult", version=2, as_frame=True)
df = adult.frame.copy()
df.head()


**Question:** Is this a classification or regression problem?  
**Target column:** ???


### Choose columns & split FIRST

In [ ]:
# 2) Choose columns (keep it small for speed)
numeric_features = ["age", "hours-per-week", "education-num", "capital-gain", "capital-loss"]
categorical_features = ["workclass", "marital-status", "occupation", "sex", "native-country"]
target_col = "class"

use_cols = numeric_features + categorical_features + [target_col]
df = df[use_cols].copy()

# 3) Split FIRST (to avoid leakage)
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
X_train.shape, X_test.shape


**Why split first?**  
So that ....


### Handle missing values

In [ ]:
# 4) Impute missing values
# Numeric → mean (try median later if you like)
num_imputer = SimpleImputer(strategy="mean")
X_train_num = pd.DataFrame(num_imputer.fit_transform(X_train[numeric_features]),
                           columns=numeric_features, index=X_train.index)
X_test_num  = pd.DataFrame(num_imputer.transform(X_test[numeric_features]),
                           columns=numeric_features, index=X_test.index)

# Categorical → most frequent
cat_imputer = SimpleImputer(strategy="most_frequent")
X_train_cat_raw = pd.DataFrame(cat_imputer.fit_transform(X_train[categorical_features]),
                               columns=categorical_features, index=X_train.index)
X_test_cat_raw  = pd.DataFrame(cat_imputer.transform(X_test[categorical_features]),
                               columns=categorical_features, index=X_test.index)


### Encode categoricals

In [ ]:
# 5) Encode categoricals
# Label-encode ONE column ('sex'); one-hot the rest
label_encode_col = "sex"

# Copies
X_train_processed = X_train_cat_raw.copy()
X_test_processed  = X_test_cat_raw.copy()

# LabelEncoder: fit on train, apply to test
le = LabelEncoder()
X_train_processed[label_encode_col] = le.fit_transform(X_train_processed[label_encode_col])
X_test_processed[label_encode_col]  = le.transform(X_test_processed[label_encode_col])

# One-hot all other categorical columns
onehot_cols = [c for c in X_train_processed.columns if c != label_encode_col]

# Version-safe: sparse_output (>=1.2) vs sparse (<1.2)
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

X_train_ohe_array = ohe.fit_transform(X_train_processed[onehot_cols])
X_test_ohe_array  = ohe.transform(X_test_processed[onehot_cols])

ohe_feature_names = ohe.get_feature_names_out(onehot_cols)
X_train_ohe = pd.DataFrame(X_train_ohe_array, columns=ohe_feature_names)
X_test_ohe  = pd.DataFrame(X_test_ohe_array,  columns=ohe_feature_names)

# Combine label-encoded + one-hot (reset indices for alignment)
label_encoded_train = X_train_processed[[label_encode_col]].reset_index(drop=True)
label_encoded_test  = X_test_processed[[label_encode_col]].reset_index(drop=True)
X_train_cat = pd.concat([label_encoded_train, X_train_ohe.reset_index(drop=True)], axis=1)
X_test_cat  = pd.concat([label_encoded_test,  X_test_ohe.reset_index(drop=True)],  axis=1)

X_train_cat.shape, X_test_cat.shape


**Reflect:**  
- When might **label encoding** be risky?  
- Why is **one-hot** often safer for models like Logistic Regression or Neural Nets?


### Scale numeric features (practice)

In [ ]:
# 6) Scale numeric features (practice—even if RF doesn’t need it)
scaler = StandardScaler()
X_train_num_scaled = pd.DataFrame(scaler.fit_transform(X_train_num), columns=numeric_features)
X_test_num_scaled  = pd.DataFrame(scaler.transform(X_test_num),   columns=numeric_features)

# Assemble final matrices (reset indices to align rows)
X_train_final = pd.concat([X_train_num_scaled.reset_index(drop=True),
                           X_train_cat.reset_index(drop=True)], axis=1)
X_test_final  = pd.concat([X_test_num_scaled.reset_index(drop=True),
                           X_test_cat.reset_index(drop=True)],  axis=1)

X_train_final.shape, X_test_final.shape


**When would you prefer**:
- `MinMaxScaler` ?
- `RobustScaler` ?


### Train RF + Evaluate

In [ ]:
# 7) Train a baseline Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_final, y_train)

y_pred = rf.predict(X_test_final)
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred, labels=rf.classes_)
sns.heatmap(pd.DataFrame(cm, index=[f"true_{c}" for c in rf.classes_],
                            columns=[f"pred_{c}" for c in rf.classes_]),
            annot=True, fmt="d", cmap="Blues")
plt.show()


### Deep dive: RF Object

In [ ]:
# 8) Inspect the RF object: attributes & methods
print("Number of trees:", rf.n_estimators)
print("Classes:", rf.classes_)
print("Max depth setting:", rf.max_depth)

# Feature importances (top 10)
importances = pd.Series(rf.feature_importances_, index=X_train_final.columns).sort_values(ascending=False)
importances.head(10)


In [ ]:
# Predict vs predict_proba
pred_label = rf.predict(X_test_final[:5])
pred_prob  = rf.predict_proba(X_test_final[:5])
pred_label, pred_prob


**Explain in your own words:**  
- What’s the difference between `predict` and `predict_proba`?  
- Which would you show in an app UI, and why?


### Tinkering with Hyperparameters

In [ ]:
# 9) Tinker: try one change, re-run report, log in experiment_log.md
rf2 = RandomForestClassifier(
    n_estimators=200,   # try 50 / 100 / 200
    max_depth=None,     # try 10 / 20 / None
    min_samples_split=2,  # try 2 / 5
    random_state=42
)
rf2.fit(X_train_final, y_train)
print(classification_report(y_test, rf2.predict(X_test_final)))


### GridSearchCV

In [ ]:
# 10)
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [None, 10, 20],
}
grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid.fit(X_train_final, y_train)

print("Best params:", grid.best_params_)
print("Best CV accuracy:", round(grid.best_score_, 4))
print("Test accuracy with best params:", round(grid.best_estimator_.score(X_test_final, y_test), 4))


# 🏠 Homework: GridSearchCV vs RandomizedSearchCV

Today we used **GridSearchCV** to systematically test a small parameter grid.  
But GridSearch gets expensive if the parameter space is large.  
An alternative is **RandomizedSearchCV**: it samples combinations at random.

**Task:**  
1. Run the provided code that compares GridSearchCV and RandomizedSearchCV.  
2. Note which one is faster, and whether they found similar/better hyperparameters.  
3. Add your reflection in `students/experiment_log.md` under "Run 2".


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define parameter distributions for RandomizedSearch
param_dist = {
    "n_estimators": randint(50, 300),
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": randint(2, 10)
}

# GridSearch (small grid)
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid, cv=3, scoring="accuracy", n_jobs=-1)

random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42),
                                   param_dist, n_iter=10, cv=3,
                                   scoring="accuracy", n_jobs=-1,
                                   random_state=42)

print("Running GridSearchCV...")
grid.fit(X_train_final, y_train)

print("Running RandomizedSearchCV...")
random_search.fit(X_train_final, y_train)

print("Best params (GridSearch):", grid.best_params_)
print("Best score (GridSearch):", round(grid.best_score_, 4))

print("Best params (RandomizedSearch):", random_search.best_params_)
print("Best score (RandomizedSearch):", round(random_search.best_score_, 4))


## Reflection Questions
- Which method finished faster? Why?  
- Did they find similar or different best parameters?  
- When would you choose GridSearchCV vs RandomizedSearchCV in practice?  


# 🎛️ Turn Your Random Forest Into a Gradio App

**Goal:** Wrap our already-trained Random Forest (RF) into a tiny web app so anyone can try inputs and see predictions (and confidence).

**What you should already have (from the earlier notebook):**
- A trained `rf` model.
- Preprocessing objects: `num_imputer`, `cat_imputer`, `le` (LabelEncoder for `"sex"`), `ohe`, `scaler`.
- Feature lists: `numeric_features`, `label_encode_col = "sex"`, `onehot_cols`.
- The final training matrix `X_train_final` (so we can keep the exact column order).

> If you get “name is not defined” errors later, scroll up and re-run the cells that define/train these.





---

## Step 1 — Lock in the feature order
We must feed the app’s processed row to the model in the **exact** column order used during training.

**Action:** Create `feature_order = list(X_train_final.columns)` in a code cell.

**Why:** Prevents misalignment (wrong columns → wrong predictions).



In [ ]:
feature_order = list(X_train_final.columns)  # numeric (scaled) + categorical block


In [ ]:
feature_order

---

## Step 2 — Install & import Gradio
We’ll use Gradio to make a small UI with sliders/dropdowns.

**Action:**  
- `!pip -q install gradio`  
- `import gradio as gr` (+ `pandas`, `numpy` if needed)

**Why:** Gradio handles the interface so we focus on ML logic.



In [ ]:
!pip -q install gradio
import gradio as gr
#import pandas as pd
#import numpy as np


---

## Step 3 — Write a `preprocess_input(user_dict)` helper
This function should:
1. Build a one-row `DataFrame` from the raw UI inputs.
2. **Numeric**: impute with `num_imputer` → scale with `scaler`.
3. **Categorical**: impute with `cat_imputer` → label-encode the `"sex"` column with `le` → one-hot the rest with `ohe`.
4. Concatenate numeric + categorical.
5. Add any missing columns (fill with 0) and **reorder** using `feature_order`.
6. Return a 1-row `DataFrame` ready for `rf.predict` / `rf.predict_proba`.

**Why:** The app must apply **the same preprocessing** as training, learned on the train split only.



In [ ]:
def preprocess_input(user_dict):
    """
    Apply the SAME transforms as train-time to a single-row user_dict and
    return a 1-row DataFrame with columns exactly matching feature_order.
    """
    # 1) Raw one-row frame
    df_in = pd.DataFrame([user_dict])

    # 2) Numeric -> impute -> scale
    Xn = pd.DataFrame(num_imputer.transform(df_in[numeric_features]),
                      columns=numeric_features)
    Xn_scaled = pd.DataFrame(scaler.transform(Xn), columns=numeric_features)

    # 3) Categorical -> impute -> label-encode ONE col -> one-hot the rest
    df_cat = df_in[[label_encode_col] + onehot_cols].copy()
    df_cat = pd.DataFrame(cat_imputer.transform(df_cat),
                          columns=[label_encode_col] + onehot_cols)

    # Label-encode the chosen column (assumes no unseen categories)
    df_cat[label_encode_col] = le.transform(df_cat[label_encode_col])


    # one-hot the remaining categoricals (trained with handle_unknown="ignore")
    Xc_ohe = pd.DataFrame(
        ohe.transform(df_cat[onehot_cols]),
        columns=ohe.get_feature_names_out(onehot_cols)
    )

    # final categorical block = label-encoded col + one-hot block
    Xc_final = pd.concat(
        [df_cat[[label_encode_col]].reset_index(drop=True),
         Xc_ohe.reset_index(drop=True)],
        axis=1
    )

    # 4) Assemble numeric + categorical
    X_final = pd.concat(
        [Xn_scaled.reset_index(drop=True), Xc_final.reset_index(drop=True)],
        axis=1
    )

    # 5) Enforce EXACT training columns & order in ONE step
    # - drops any extra columns
    # - inserts any missing one-hot columns filled with 0
    #X_final = X_final.reindex(columns=feature_order, fill_value=0)
    # at end of preprocess_input
    expected_cols = list(getattr(rf, "feature_names_in_", feature_order))
    X_final = X_final.reindex(columns=expected_cols, fill_value=0)
    return X_final


    #return X_final


---

## Step 4 — Write the `predict(...)` function
Keep the UI compact (start with 5 inputs). Good defaults for the rest are fine.

**Inputs to expose (suggested):**
- `age` (slider)
- `hours_per_week` (slider)
- `education_num` (slider)
- `sex` (radio from training categories)
- `workclass` (dropdown from training categories)

**Inside `predict`:**
- Build a `raw` dict from those inputs (plus reasonable defaults for other features).
- Call `preprocess_input(raw)` → `Xf`.
- Use `rf.predict(Xf)` for the label.
- If available, use `rf.predict_proba(Xf)` and show **P(>50K)**.

**Why:** Labels are decisive; probabilities show **confidence** and open the door to threshold choices.



In [ ]:
# Simple prediction function returning label + probability of >50K if available
def predict(age, hours_per_week, education_num, sex, workclass):
    raw = {
        "age": age,
        "hours-per-week": hours_per_week,
        "education-num": education_num,
        "sex": sex,               # label-encoded column
        "workclass": workclass,   # one-hot
        # Defaults for demo (you can expose these later in the UI)
        "marital-status": "Never-married",
        "occupation": "Adm-clerical",
        "native-country": "United-States",
        "capital-gain": 0,
        "capital-loss": 0,
    }
    Xf = preprocess_input(raw)
    label = rf.predict(Xf)[0]
    out = f"Prediction: {label}"
    if hasattr(rf, "predict_proba"):
        classes = list(rf.classes_)
        proba = rf.predict_proba(Xf)[0]
        # Show the probability of >50K if that class exists
        if ">50K" in classes:
            p = proba[classes.index(">50K")]
            out += f"  |  P(>50K) = {p:.2f}"
    return out


---

## Step 5 — Build the UI and launch
- Pull choices for `sex` and `workclass` from the **training data** (so no unseen categories).
- Create a `gr.Interface(fn=predict, inputs=[...], outputs="text")`.
- Call `demo.launch(share=True)` (use `share=False` if tunnels are blocked).




In [ ]:
# Use categories from your training set (already imputed)
workclass_choices = sorted(pd.unique(X_train["workclass"].dropna()))
sex_choices = sorted(pd.unique(X_train["sex"].dropna()))

demo = gr.Interface(
    fn=predict,
    inputs=[
        gr.Slider(17, 90, value=37, step=1, label="age"),
        gr.Slider(1, 80, value=40, step=1, label="hours_per_week"),
        gr.Slider(1, 16, value=10, step=1, label="education_num"),
        gr.Radio(list(sex_choices), label="sex"),
        gr.Dropdown(list(workclass_choices), label="workclass"),
    ],
    outputs=gr.Textbox(label="Model Output"),
    title="Income Classifier (Random Forest)",
    description="Baseline RF wrapped as a Gradio app. Shows label and P(>50K)."
)

demo.launch(share=True, show_error=True)  # use share=False if school network blocks external tunnels


---

## 🌟 Stretch Challenge (pick one)

### A) Add more inputs
Expose `marital-status` and/or `occupation` in the UI. Update `predict` to pass them through `preprocess_input`.

**Hint:** Keep categorical choices from the training set to avoid unseen-label errors.

---

### B) Show **feature importance** (top-k)
After training, RF has `rf.feature_importances_`. Let users click a button to see the **top 10 features** your model relied on.

**How:**
- Create a small function that returns a table or text of the top-k from:
  - `pd.Series(rf.feature_importances_, index=X_train_final.columns).sort_values(ascending=False).head(10)`

**Optional UI idea:**
- Add a secondary `gr.Interface` or a `gr.Tab` / `gr.Accordion` that calls this function (e.g., `outputs="dataframe"` or text).
- Ask: *Does importance change if you retrain with different hyperparameters?*

> Note: RF importances are heuristic; they’re useful for discussion, not a complete story (advanced: SHAP later).

---

### C) Precision vs Recall slider (advanced)
Let the user pick a **threshold** for classifying `>50K` using `predict_proba`. Show how higher thresholds **increase precision** but **decrease recall**.

**How:**
- Add a slider 0.0–1.0 (“Decision Threshold for >50K”).
- If `P(>50K) ≥ threshold` → predict `>50K`, else `<=50K`.

**Discuss:** When would you prefer high precision vs high recall in a real app?

---

## Troubleshooting Tips
- **“unseen label” errors**: Make sure UI choices (e.g., `sex`, `workclass`) come from training categories.  
- **Wrong feature order**: Always reorder with `feature_order` before predicting.  
- **Variables not defined**: Re-run training/preprocessing cells above or import from your previous notebook.

---

### Exit Ticket (today)
- Launch your app and test 3 scenarios.
- Paste one app screenshot + a 2–3 sentence reflection in `students/experiment_log.md`.